## Contents:
- Complexity summary table
- Singly Linked List: Node, LinkedList classes with full set of operations
- Doubly Linked List
- Circular Linked List
- Common algorithms: reverse (iterative/recursive), detect cycle (Floyd), merge two sorted lists,
remove duplicates, find middle, kth from end, palindrome check, sorting via merge sort on lists
- Simple unit-test / usage examples at bottom


This file is written for learning: read the docstrings and run the examples.


In [2]:
from __future__ import annotations
from typing import Any, Optional, Iterable, Generator, Tuple


# --------------------------------------
# Complexity summary (Singly Linked List)
# Operation | Time Complexity
# ------------------------------------------------
# insert_head | O(1)
# insert_tail | O(n) (or O(1) with tail pointer)
# insert_at(index) | O(n)
# delete_head | O(1)
# delete_tail | O(n)
# delete_value | O(n)
# search | O(n)
# reverse | O(n)
# find_middle | O(n)
# kth_from_end | O(n)
# detect_cycle | O(n)
# merging two sorted lists | O(n+m)
# ------------------------------------------------

In [4]:
class Node:
# """Node for singly linked list.
    def __init__(self, data, nxt=None):
        self.data = data
        self.next = nxt


    def __repr__(self):
        return f"Node({self.data!r})"

Singly linked list with helpful methods and iterator support.


Supports:
- insert_head, insert_tail, insert_at(index)
- delete_head, delete_tail, delete_value
- search, reverse (iterative & recursive), size, to_list
- find_middle, kth_from_end, detect_cycle
- merge_sorted (staticmethod)

In [6]:
class LinkedList:
    def __init__(self, iterable=None):
        self.head = None
        self._tail = None # maintain tail for O(1) tail inserts
        self._size = 0
        if iterable:
            for item in iterable:
                self.insert_tail(item)
    def __len__(self):
        return self._size
    def __iter__(self):
        cur=self.head
        while cur:
            yield cur.data
            cur=cur.next
        
    def __repr__(self):
        return "LinkedList([" + ", ".join(repr(x) for x in self) + "])"
# --- Basic inserts ---

    def insert_head(self, data):
        """Insert a new node with the given data at the head of the list.
        Time Complexity: O(1)
        """
        new_node = Node(data, self.head)
        self.head = new_node
        if self._size == 0:
            self._tail = new_node
        self._size += 1
    def insert_tail(self, data):
        """Insert a new node with the given data at the tail of the list.
        Time Complexity: O(1) with tail pointer, O(n) without.
        """
        new_node = Node(data)
        if self._size == 0:
            self.head = new_node
            self._tail = new_node
        else:
            self._tail.next = new_node
            self._tail = new_node
        self._size += 1
    def insert_at(self, index, data):
        """Insert a new node with the given data at the specified index.
        If index is greater than the size of the list, insert at the tail.
        Time Complexity: O(n)
        """
        if index <= 0:
            self.insert_head(data)
        elif index >= self._size:
            self.insert_tail(data)
        else:
            cur = self.head
            for _ in range(index - 1):
                cur = cur.next
            new_node = Node(data, cur.next)
            cur.next = new_node
            self._size += 1
# --- Basic deletes ---
    def delete_head(self):
        """Delete the head node of the list and return its data.
        Raises IndexError if the list is empty.
        Time Complexity: O(1)
        """
        if self._size == 0:
            raise IndexError("delete_head from empty list")
        removed_data = self.head.data
        self.head = self.head.next
        self._size -= 1
        if self._size == 0:
            self._tail = None
        return removed_data
    def delete_tail(self):
        """Delete the tail node of the list and return its data.
        Raises IndexError if the list is empty.
        Time Complexity: O(n)
        """
        if self._size == 0:
            raise IndexError("delete_tail from empty list")
        if self._size == 1:
            return self.delete_head()
        cur = self.head
        while cur.next != self._tail:
            cur = cur.next
        removed_data = self._tail.data
        cur.next = None
        self._tail = cur
        self._size -= 1
        return removed_data
    def delete_value(self, value):
        """Delete the first node with the specified value.
        Raises ValueError if the value is not found.
        Time Complexity: O(n)
        """
        if self._size == 0:
            raise ValueError("delete_value from empty list")
        if self.head.data == value:
            return self.delete_head()
        cur = self.head
        while cur.next and cur.next.data != value:
            cur = cur.next
        if cur.next is None:
            raise ValueError(f"{value} not found in list")
        removed_data = cur.next.data
        if cur.next == self._tail:
            self._tail = cur
        cur.next = cur.next.next
        self._size -= 1
        return removed_data
# --- search ---
    def search(self, value):
        """Search for the first node with the specified value.
        Returns the node if found, else None.
        Time Complexity: O(n)
        """
        cur = self.head
        while cur:
            if cur.data == value:
                return cur
            cur = cur.next
        return None
# --- Utilities ---
    
    def reverse__iterative(self):
        """Reverse the linked list in place (iterative version).
        Time Complexity: O(n)
        """
        prev = None
        cur = self.head
        self._tail = self.head
        while cur:
            nxt = cur.next
            cur.next = prev
            prev = cur
            cur = nxt
        self.head = prev
    def reverse__recursive(self):
        """Reverse the linked list in place (recursive version).
        Time Complexity: O(n)
        """
        def _reverse_rec(cur, prev):
            if not cur:
                return prev
            nxt = cur.next
            cur.next = prev
            return _reverse_rec(nxt, cur)
        self._tail = self.head
        self.head = _reverse_rec(self.head, None)
    def find_middle(self):
        slow=fast=self.head
        while fast and fast.next:
            slow=slow.next
            fast=fast.next.next
        return None if slow is None else slow.data
    def kth_from_end(self, k):
        if k <= 0 or k > self._size:
            raise IndexError("k is out of bounds")
        slow=fast=self.head
        for _ in range(k):
            fast=fast.next
        while fast:
            slow=slow.next
            fast=fast.next
        return slow.data
    def detect_cycle(self):
        slow=fast=self.head
        while fast and fast.next:
            slow=slow.next
            fast=fast.next.next
            if slow==fast:
                return True
        return False
    def to_list(self):
        return list(self)
    
    @staticmethod
    def merge_sorted(list1, list2):
        """Merge two sorted linked lists into a new sorted linked list.
        Time Complexity: O(n + m)
        """
        merged = LinkedList()
        cur1, cur2 = list1.head, list2.head
        while cur1 and cur2:
            if cur1.data <= cur2.data:
                merged.insert_tail(cur1.data)
                cur1 = cur1.next
            else:
                merged.insert_tail(cur2.data)
                cur2 = cur2.next
        while cur1:
            merged.insert_tail(cur1.data)
            cur1 = cur1.next
        while cur2:
            merged.insert_tail(cur2.data)
            cur2 = cur2.next
        return merged   




double LinkedList


In [7]:
class DNode:
    def __init__(self, data, prev=None, nxt=None):
        self.data = data
        self.prev = prev
        self.next = nxt


    def __repr__(self):
        return f"DNode({self.data!r})"

In [13]:
class DoublyLinkedList:
    def __init__(self, iterable=None):
        self.head = None
        self.tail = None
        self._size = 0
        if iterable:
            for item in iterable:
                self.insert_tail(item)
    def __len__(self):
        return self._size
    def __iter__(self):
        cur=self.head
        while cur:
            yield cur.data
            cur=cur.next
        
    def __repr__(self):
        return "DoublyLinkedList([" + ", ".join(repr(x) for x in self) + "])"
# --- Basic inserts ---
    def insert_head(self, data):
        new_node = DNode(data, None, self.head)
        if self.head:
            self.head.prev = new_node
        self.head = new_node
        if self._size == 0:
            self.tail = new_node
        self._size += 1
    def insert_tail(self, data):
        new_node = DNode(data, self.tail, None)
        if self.tail:
            self.tail.next = new_node
        self.tail = new_node
        if self._size == 0:
            self.head = new_node
        self._size += 1
    def insert_at(self, index, data):
        if index <= 0:
            self.insert_head(data)
        elif index >= self._size:
            self.insert_tail(data)
        else:
            cur = self.head
            for _ in range(index):
                cur = cur.next
            new_node = DNode(data, cur.prev, cur)
            if cur.prev:
                cur.prev.next = new_node
            cur.prev = new_node
            self._size += 1
# --- Basic deletes ---
    def delete_head(self):  
        if self._size == 0:
            raise IndexError("delete_head from empty list")
        removed_data = self.head.data
        self.head = self.head.next
        if self.head:
            self.head.prev = None
        self._size -= 1
        if self._size == 0:
            self.tail = None
        return removed_data
    def delete_tail(self):
        if self._size == 0:
            raise IndexError("delete_tail from empty list")
        removed_data = self.tail.data
        self.tail = self.tail.prev
        if self.tail:
            self.tail.next = None
        self._size -= 1
        if self._size == 0:
            self.head = None
        return removed_data
    def delete_value(self, value):
        if self._size == 0:
            raise ValueError("delete_value from empty list")
        cur = self.head
        while cur and cur.data != value:
            cur = cur.next
        if cur is None:
            raise ValueError(f"{value} not found in list")
        if cur.prev:
            cur.prev.next = cur.next
        if cur.next:
            cur.next.prev = cur.prev
        if cur == self.head:
            self.head = cur.next
        if cur == self.tail:
            self.tail = cur.prev
        self._size -= 1
        return cur.data
# --- search ---
    def search(self, value):
        cur = self.head
        while cur:
            if cur.data == value:
                return cur
            cur = cur.next
        return None
# --- Utilities ---
    def to_list(self):
        out = []
        cur = self.head
        while cur:
            out.append(cur.data)
            cur = cur.next
        return out   

In [18]:
class CircularLinkedList:
    def __init__(self):
        self.head = None
        self._size = 0
    
    def __len__(self):
        return self._size
    def __iter__(self):
        if not self.head:
            return
        cur = self.head
        for _ in range(self._size):
            yield cur.data
            cur = cur.next
    def __repr__(self):
        return "CircularLinkedList([" + ", ".join(repr(x) for x in self) + "])"
# --- Basic inserts ---
    def insert_head(self, data):
        new_node = Node(data)
        if self._size == 0:
            new_node.next = new_node
            self.head = new_node
        else:
            new_node.next = self.head
            tail = self.head
            for _ in range(self._size - 1):
                tail = tail.next
            tail.next = new_node
            self.head = new_node
        self._size += 1
    def insert_tail(self, data):
        new_node = Node(data)
        if self._size == 0:
            new_node.next = new_node
            self.head = new_node
        else:
            tail = self.head
            for _ in range(self._size - 1):
                tail = tail.next
            tail.next = new_node
            new_node.next = self.head
        self._size += 1
    def insert_at(self, index, data):
        if index <= 0:
            self.insert_head(data)
        elif index >= self._size:
            self.insert_tail(data)
        else:
            cur = self.head
            for _ in range(index - 1):
                cur = cur.next
            new_node = Node(data, cur.next)
            cur.next = new_node
            self._size += 1
# --- Basic deletes ---
    def delete_head(self):
        if self._size == 0:
            raise IndexError("delete_head from empty list")
        removed_data = self.head.data
        if self._size == 1:
            self.head = None
        else:
            tail = self.head
            for _ in range(self._size - 1):
                tail = tail.next
            self.head = self.head.next
            tail.next = self.head
        self._size -= 1
        return removed_data
    def delete_tail(self):
        if self._size == 0:
            raise IndexError("delete_tail from empty list")
        if self._size == 1:
            return self.delete_head()
        cur = self.head
        for _ in range(self._size - 2):
            cur = cur.next
        removed_data = cur.next.data
        cur.next = self.head
        self._size -= 1
        return removed_data
    def delete_value(self, value):
        if self._size == 0:
            raise ValueError("delete_value from empty list")
        if self.head.data == value:
            return self.delete_head()
        cur = self.head
        for _ in range(self._size - 1):
            if cur.next.data == value:
                break
            cur = cur.next
        else:
            raise ValueError(f"{value} not found in list")
        removed_data = cur.next.data
        cur.next = cur.next.next
        self._size -= 1
        return removed_data
# --- search ---
    def search(self, value):
        if self._size == 0:
            return None
        cur = self.head
        for _ in range(self._size):
            if cur.data == value:
                return cur
            cur = cur.next
        return None 
# --- Utilities ---
    def to_list(self):
        return list(self)
    


In [11]:


import time

# Prepare inputs
LL_best = LinkedList([10, 20, 30, 40, 50])   # target at head
LL_avg  = LinkedList(list(range(1000)))      # target in middle ~ 500
LL_worst = LinkedList(list(range(10000)))    # target at tail (or not present)

# Targets
target_best = 10
target_avg  = 500
target_worst = 9999   # present at tail; change to -1 for not-present worst-case too

# Measure simple single-run times (for demonstration)
def time_search(ll, tgt):
    t0 = time.perf_counter()
    idx = ll.search(tgt)
    t1 = time.perf_counter()
    print(f"search({tgt}) -> idx={idx}  time={(t1-t0):.6f}s")

print("Singly LinkedList — best / avg / worst")
time_search(LL_best, target_best)       # O(1)
time_search(LL_avg, target_avg)         # ~O(n/2)
time_search(LL_worst, target_worst)     # ~O(n)


Singly LinkedList — best / avg / worst
search(10) -> idx=Node(10)  time=0.000002s
search(500) -> idx=Node(500)  time=0.000031s
search(9999) -> idx=Node(9999)  time=0.000671s


In [17]:
import time

# Prepare inputs
DLL_best = DoublyLinkedList([10, 20, 30, 40, 50])         # target at head
DLL_avg  = DoublyLinkedList(list(range(2000)))       # target in middle ~ 1000
DLL_worst = DoublyLinkedList(list(range(5000)))      # we will delete tail

# Targets
target_best = 1
target_avg  = 1000

# Measure simple single-run times (for demonstration)
def time_search(dll, tgt):
    t0 = time.perf_counter()
    node = dll.search(tgt)
    t1 = time.perf_counter()
    print(f"search({tgt}) -> idx={node}  time={(t1-t0):.6f}s")

print("Doubly LinkedList — best / avg / worst")
time_search(DLL_best, target_best)   # O(1)
time_search(DLL_avg, target_avg)     # ~O(n/2)

# Worst-case demonstration using delete_tail (O(1) in DLL, unlike singly)
print("\nDelete tail demo:")
print("tail before delete:", DLL_worst.tail)
t0 = time.perf_counter()
removed = DLL_worst.delete_tail()
t1 = time.perf_counter()
print(f"delete_tail() -> removed={removed}  time={(t1-t0):.6f}s")
print("tail after delete:", DLL_worst.tail)


Doubly LinkedList — best / avg / worst
search(1) -> idx=None  time=0.000003s
search(1000) -> idx=DNode(1000)  time=0.000080s

Delete tail demo:
tail before delete: DNode(4999)
delete_tail() -> removed=4999  time=0.000043s
tail after delete: DNode(4998)


In [21]:
import time

# ---- Prepare test lists ----
C_best = CircularLinkedList()
for x in [100, 200, 300, 400]:
    C_best.insert_tail(x)          # small list, target near head

C_avg = CircularLinkedList()
for x in range(1500):
    C_avg.insert_tail(x)           # medium list, target in middle

C_worst = CircularLinkedList()
for x in range(8000):
    C_worst.insert_tail(x)         # large list, target near tail / not present


# ---- Timing wrapper ----
def time_search(cll, tgt, label):
    t0 = time.perf_counter()
    node = cll.search(tgt)
    t1 = time.perf_counter()
    found = node.data if node else None
    print(f"{label}: search({tgt}) -> {found}, time={(t1 - t0):.6f}s")


# ---- Run cases ----
print("CircularLinkedList — best / avg / worst")

time_search(C_best, 100,  "Best")      # O(1), near head
time_search(C_avg, 750,  "Average")    # ~O(n/2), middle element
time_search(C_worst, 7999, "Worst")    # ~O(n), last element

# Optional: not found case (true worst)
time_search(C_worst, -1,  "NotFound")  # traverses whole circle


CircularLinkedList — best / avg / worst
Best: search(100) -> 100, time=0.000003s
Average: search(750) -> 750, time=0.000023s
Worst: search(7999) -> 7999, time=0.000230s
NotFound: search(-1) -> None, time=0.000336s
